In [ ]:
import duckdb

In [ ]:
con = duckdb.connect(database='dados_duckdb.db', read_only=False)

In [ ]:
df = con.execute("""
                 SELECT *
                 FROM (
                   SELECT * , ROW_NUMBER() OVER (PARTITION BY NATBR ORDER BY data_ingestao DESC) AS row
                   FROM bronze_z0019
                   WHERE data_ingestao >= '2026-01-05'
                 ) WHERE row=1
                 """).fetchdf()      
df.head(10)

In [ ]:
# Realizando exclusões de colunas e renomeações
df_final =df.drop(columns=['nome_arquivo','data_ingestao','row'])
df_final = df_final.rename(columns={"NATBR":"id"})
df_final = df_final.rename(columns={"MAKTX":"nm_produto"})
df_final = df_final.rename(columns={"WERKS":"id_categoria"})
df_final = df_final.rename(columns={"MAINS":"id_fornecedor"})
df_final = df_final.rename(columns={"LABST":"vl_preço"})
df_final.head(10)

In [ ]:
# Como está tipado cada uma das colunas
df_final.dtypes

In [ ]:
# Alterando os tipos de dados das colunas
df2 = df_final
df2 = df2.astype(
    {
        "id": int,
        "nm_produto": str,
        "id_categoria": str,
        "id_fornecedor": int,
        "vl_preço": float
    }
)
df2.dtypes

In [ ]:
con.execute("""
            CREATE TABLE IF NOT EXISTS produtos (
            id BIGINT,
            nm_produto TEXT,
            id_categoria TEXT,
            id_fornecedor BIGINT,
            vl_preço FLOAT
             )
""")

In [ ]:
df2.head(10)

In [ ]:
# Ingestão dos dados tratados para a tabela de produtos
con.execute("INSERT INTO produtos SELECT * FROM df2")

In [ ]:
# Executando o carregamento dos dados tratados para a tabela de produtos
df_resultado = con.execute("SELECT * FROM produtos").fetchdf()
df_resultado.head(10)

In [ ]:
#con.execute("DROP TABLE produtos")

In [ ]:
con.close()